In [491]:
import pandas as pd  # import the neccessary libraries
pd.set_option('display.max_columns', 300)  # display all the columns
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

plt.style.use('seaborn')
sns.set(style="white")

## Step 1: Read in hold out data, scalers, and best model

In [493]:
df = pd.read_csv('kc_house_data_test_features.csv')# read in the csv file with holdout data
zip_df = pd.read_csv('zip_code_extra_data.csv')
print(df.shape)  # check the dimensions of the model
df.head()   # print the first 5 rows

(4323, 21)


,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [494]:
pickle_in = open('model.pickle','rb')
final_model = pickle.load(pickle_in)

In [495]:
pickle_in = open('scaler.pickle','rb')
final_scaler = pickle.load(pickle_in)

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [496]:
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [497]:
# transformed_holdout = final_scaler(holdout)

zip_df = pd.read_csv('zip_code_extra_data.csv')
##### Merge Zipcode Data #####
df = pd.merge(df,zip_df,on='zipcode')
#### Drop Duplicates, keeping most recent sold ####
df['year_sold'] = df['date'].str[0:4].astype(int)
df = df.sort_values(by=['year_sold']).drop_duplicates(['id'],keep='last')
#### Dropping columns I dont intend to Use ####
df.drop(columns=['full_time_earnings','part_time_earnings','no_earnings','car_truck_van',
                 'public_transportation','taxi','motorcycle',
       'bike_walk_other', 'work_home', 'less_than_hs', 'high_school_grad',
       'associates', 'bachelors', 'masters', 'professional_school_degree',
       'doctorate', 'public_school_enrollment', 'private_school_enrollment',
       'no_school_enrollment', 'for_rent', 'rented_and_unoccupied',
       'for_sale_only', 'sold_and_unoccupied', 'seasonal_or_rec_use',
       'migrant_worker_housing', 'vacant_other_reasons',
       'in_occupied_housing_units', 'adult_correctional_facility',
       'juvenile_facilities', 'nursing_facilities', 'other_institutional',
       'military_quarters', 'other_noninstitutional',
       'house_owned_with_mortgage', 'house_owned_outright',
       'renter_occupied_housing', 'vacant_households',
       'husband_and_wife_house', 'single_guardian', 'singles',
       'singles_with_roommate', 'male', 'female','male_median_age', 'female_median_age',
         'total_housing_units','land_area', 'water_area_sq_mile',
       'occupied_housing_units'], inplace=True)


In [498]:
#### Drop Duplicates, keeping most recent sold ####
df['year_sold'] = df['date'].str[0:4].astype(int)
df = df.sort_values(by=['year_sold']).drop_duplicates(['id'],keep='last')

##### Initial Data Cleaning -- Based Mostly off Abnormal Values

#Add column bath/bed
df['bath_per_bed'] = np.where(df['bedrooms'] > 0, df['bathrooms'] / df['bedrooms'], 0)
df.drop(columns=['bathrooms'], inplace=True)
#Add Log_Price
#df['log_price'] = np.where(df['price'], np.log(df['price']), df['price'])
#Condesing 33 and 11 Bedroom values to 10
df['bedrooms'] = np.where(df['bedrooms'] > 10, 3, df['bedrooms'])
#House with 0 bedrooms to the Mean Bedrooms
df['bedrooms'] = np.where(df['bedrooms'] == 0, 3, df['bedrooms'])
##### Convert Date to Year_Sold and Month_Sold ####
df['year_sold'] = df['date'].str[0:4].astype(int)
df['month_sold'] = df['date'].str[4:6].astype(int)
df.drop(columns=['date'], inplace=True)
#### Floors seems to not be very significant, converting to binary (0 = 1 Floor, 1 = Multi-Floors)
df['floors'] = np.where(df['floors'] > 1, 1, 0)
#Drop 'sqft_above' column
df.drop(columns=['sqft_above'], inplace=True)
#Turn 'sqft_basement' into binary "Basement" column
df['basement'] = np.where(df['sqft_basement'] > 0, 1, 0)
#Drop 'sqft_basement' column
df.drop(columns=['sqft_basement'], inplace=True)
df.drop(columns=['sqft_living15'], inplace=True)

##### Add Season column #####
def to_season(x):
    if x == 12 or x == 1 or x == 2:
        return 'cold_month'
    elif x == 3 or x == 4 or x == 5:
        return "warm_month"
    elif x == 6 or x == 7 or x == 8:
        return "warm_month"
    else:
        return 'cold_month'
df['season_sold'] = df['month_sold'].apply(to_season)
##### Get Dummies for Season & Year Sold Columns#####
df = pd.get_dummies(df, columns = ['season_sold'], drop_first= True)
df = pd.get_dummies(df, columns = ['year_sold'], drop_first= True)
#### Drop Uneeded Columns #####
df.drop(columns=['month_sold'], inplace=True)

##### Create Years Old Bins #####
df['age'] = 2020 - df['yr_built']
df.drop(columns=['yr_built'], inplace=True)
bins = [-2,25,50,100,100000]
labels = ['1','2','3','4']
df['age'] = pd.cut(df['age'], bins=bins, labels=labels)
df = pd.get_dummies(df, columns = ['age'], drop_first= True)

##### Create Years since Renovation Column #####
# After EDA showed most houses havent been renovated, changing to binary
df['renovated'] = np.where(df['yr_renovated'] > 0, 1, 0)
df.drop(columns=['yr_renovated'], inplace=True)

##### Create Number of Bedrooms Bins#####
bed_bins = [-1,2,3,5,100]
bed_labels = ['1','2','3','4']
df['bedrooms'] = pd.cut(df['bedrooms'], bins=bed_bins, labels=bed_labels)
df = pd.get_dummies(df, columns = ['bedrooms'], drop_first= True)

##### Get Dummies for Other Categorical Variables #####
def sep_grade(x):
    if x == 13:
        return 'high'
    elif x == 10 or x == 11 or x == 11:
        return "medium"
    else:
        return 'low'
df['grade'] = df['grade'].apply(sep_grade)
df = pd.get_dummies(df, columns = ['grade'], drop_first= True)
## Grade not a great determinant of Price

#Turn condition into binary (Good Condition = 1, Bad Condition = 0)
df['condition'] = np.where(df['condition'] > 2, 1, 0)

#Get Dummies for View
df = pd.get_dummies(df, columns = ['view'], drop_first= True)

In [499]:
##### Zip Codes to Bins #####
zip_cat = pd.read_csv('zip_cat.csv', index_col = 0)
df['zip_price_level'] = df['zipcode'].apply(lambda x: zip_cat.loc[x])
df = pd.get_dummies(df, columns = ['zip_price_level'], drop_first= True)

In [500]:
features_df = df.drop(columns=['id','Unnamed: 0'])
features_df

,sqft_living,sqft_lot,floors,waterfront,condition,zipcode,lat,long,sqft_lot15,median_age,population,population_density_per_sq_mile,median_home_value,median_household_income,bath_per_bed,basement,season_sold_warm_month,year_sold_2015,age_2,age_3,age_4,renovated,bedrooms_2,bedrooms_3,bedrooms_4,grade_low,grade_medium,view_1,view_2,view_3,view_4,zip_price_level_low,zip_price_level_medium,zip_price_level_very_high
2675,1480,8009,0,0,1,98032,47.3657,-122.280,7678,33,33853,2024,234700.0,48853.0,0.583333,1,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
1258,3140,9058,0,0,1,98006,47.5462,-122.154,10018,42,36364,3402,574000.0,110290.0,0.750000,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
1256,2090,45535,0,0,1,98006,47.5538,-122.191,12889,42,36364,3402,574000.0,110290.0,0.750000,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0
1242,2980,9235,0,0,1,98006,47.5513,-122.162,10046,42,36364,3402,574000.0,110290.0,0.625000,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
1244,3470,11843,0,0,1,98006,47.5513,-122.135,13247,42,36364,3402,574000.0,110290.0,0.812500,1,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,1280,1730,1,0,1,98177,47.7032,-122.360,2121,48,19030,3341,478600.0,95648.0,1.083333,1,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
2314,2075,39553,0,0,1,98038,47.3601,-122.032,9047,37,31171,469,317700.0,95043.0,0.833333,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
1458,2650,9140,0,0,1,98177,47.7772,-122.387,7800,48,19030,3341,478600.0,95648.0,0.450000,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0
2299,2420,4981,1,0,1,98038,47.3785,-122.023,5000,37,31171,469,317700.0,95043.0,0.625000,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0


In [501]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(features_df)
poly2_columns = poly_2.get_feature_names(features_df.columns)
df_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)
print(df_poly2.shape)
df_poly2.head()

(4300, 629)


,sqft_living,sqft_lot,floors,waterfront,condition,zipcode,lat,long,sqft_lot15,median_age,population,population_density_per_sq_mile,median_home_value,median_household_income,bath_per_bed,basement,season_sold_warm_month,year_sold_2015,age_2,age_3,age_4,renovated,bedrooms_2,bedrooms_3,bedrooms_4,grade_low,grade_medium,view_1,view_2,view_3,view_4,zip_price_level_low,zip_price_level_medium,zip_price_level_very_high,sqft_living^2,sqft_living sqft_lot,sqft_living floors,sqft_living waterfront,sqft_living condition,sqft_living zipcode,sqft_living lat,sqft_living long,sqft_living sqft_lot15,sqft_living median_age,sqft_living population,sqft_living population_density_per_sq_mile,sqft_living median_home_value,sqft_living median_household_income,sqft_living bath_per_bed,sqft_living basement,sqft_living season_sold_warm_month,sqft_living year_sold_2015,sqft_living age_2,sqft_living age_3,sqft_living age_4,sqft_living renovated,sqft_living bedrooms_2,sqft_living bedrooms_3,sqft_living bedrooms_4,sqft_living grade_low,sqft_living grade_medium,sqft_living view_1,sqft_living view_2,sqft_living view_3,sqft_living view_4,sqft_living zip_price_level_low,sqft_living zip_price_level_medium,sqft_living zip_price_level_very_high,sqft_lot^2,sqft_lot floors,sqft_lot waterfront,sqft_lot condition,sqft_lot zipcode,sqft_lot lat,sqft_lot long,sqft_lot sqft_lot15,sqft_lot median_age,sqft_lot population,sqft_lot population_density_per_sq_mile,sqft_lot median_home_value,sqft_lot median_household_income,sqft_lot bath_per_bed,sqft_lot basement,sqft_lot season_sold_warm_month,sqft_lot year_sold_2015,sqft_lot age_2,sqft_lot age_3,sqft_lot age_4,sqft_lot renovated,sqft_lot bedrooms_2,sqft_lot bedrooms_3,sqft_lot bedrooms_4,sqft_lot grade_low,sqft_lot grade_medium,sqft_lot view_1,sqft_lot view_2,sqft_lot view_3,sqft_lot view_4,sqft_lot zip_price_level_low,sqft_lot zip_price_level_medium,sqft_lot zip_price_level_very_high,floors^2,floors waterfront,floors condition,floors zipcode,floors lat,floors long,floors sqft_lot15,floors median_age,floors population,floors population_density_per_sq_mile,floors median_home_value,floors median_household_income,floors bath_per_bed,floors basement,floors season_sold_warm_month,floors year_sold_2015,floors age_2,floors age_3,floors age_4,floors renovated,floors bedrooms_2,floors bedrooms_3,floors bedrooms_4,floors grade_low,floors grade_medium,floors view_1,floors view_2,floors view_3,floors view_4,floors zip_price_level_low,floors zip_price_level_medium,floors zip_price_level_very_high,waterfront^2,waterfront condition,waterfront zipcode,waterfront lat,waterfront long,waterfront sqft_lot15,waterfront median_age,waterfront population,waterfront population_density_per_sq_mile,waterfront median_home_value,waterfront median_household_income,waterfront bath_per_bed,waterfront basement,waterfront season_sold_warm_month,waterfront year_sold_2015,waterfront age_2,waterfront age_3,...,year_sold_2015 age_4,year_sold_2015 renovated,year_sold_2015 bedrooms_2,year_sold_2015 bedrooms_3,year_sold_2015 bedrooms_4,year_sold_2015 grade_low,year_sold_2015 grade_medium,year_sold_2015 view_1,year_sold_2015 view_2,year_sold_2015 view_3,year_sold_2015 view_4,year_sold_2015 zip_price_level_low,year_sold_2015 zip_price_level_medium,year_sold_2015 zip_price_level_very_high,age_2^2,age_2 age_3,age_2 age_4,age_2 renovated,age_2 bedrooms_2,age_2 bedrooms_3,age_2 bedrooms_4,age_2 grade_low,age_2 grade_medium,age_2 view_1,age_2 view_2,age_2 view_3,age_2 view_4,age_2 zip_price_level_low,age_2 zip_price_level_medium,age_2 zip_price_level_very_high,age_3^2,age_3 age_4,age_3 renovated,age_3 bedrooms_2,age_3 bedrooms_3,age_3 bedrooms_4,age_3 grade_low,age_3 grade_medium,age_3 view_1,age_3 view_2,age_3 view_3,age_3 view_4,age_3 zip_price_level_low,age_3 zip_price_level_medium,age_3 zip_price_level_very_high,age_4^2,age_4 renovated,age_4 bedrooms_2,age_4 bedrooms_3,age_4 bedrooms_4,age_4 grade_low,age_4 grade_medium,age_4 view_1,age_4 view_2,age_4 view_3,age_4 view_4,

In [502]:
kb_selected_columns = ['sqft_living', 'median_home_value', 'sqft_living^2',
       'sqft_living condition', 'sqft_living zipcode', 'sqft_living lat',
       'sqft_living long', 'sqft_living median_age',
       'sqft_living median_home_value', 'sqft_living median_household_income',
       'sqft_living bath_per_bed', 'sqft_living grade_medium',
       'condition median_home_value', 'zipcode median_home_value',
       'lat median_home_value', 'long median_home_value',
       'median_age median_home_value', 'median_home_value^2',
       'median_home_value bath_per_bed', 'median_home_value grade_medium']

## Step 3: Predict the holdout set

In [503]:
transformed_holdout = final_scaler.transform(df_poly2[kb_selected_columns])

In [504]:
print(transformed_holdout.shape)
transformed_holdout

(4300, 20)


array([[-0.65830388, -1.33379591, -0.54797697, ..., -1.00944535,
        -0.96102615, -0.26301871],
       [ 1.14627982,  1.21302768,  0.85038182, ...,  1.07731023,
         1.18499417, -0.26301871],
       [ 0.00482627,  1.21302768, -0.1509074 , ...,  1.07731023,
         1.18499417, -0.26301871],
       ...,
       [ 0.6136015 ,  0.49694465,  0.33308097, ...,  0.31362917,
        -0.38750738, -0.26301871],
       [ 0.36356882, -0.71078867,  0.12046085, ..., -0.66076133,
        -0.51036249, -0.26301871],
       [ 1.45066743,  0.0488298 ,  1.1852936 , ..., -0.09385343,
         0.18161348, -0.26301871]])

In [505]:
final_answers = final_model.predict(transformed_holdout)

final_df = pd.DataFrame(final_answers)

In [506]:
print(final_answers.shape)
final_answers

(4300,)


array([259441.87297777, 897534.93689507, 694180.40298153, ...,
       678977.18644963, 335031.33163518, 569796.69585679])

In [507]:
final_df.describe()

,0
count,4.300000e+03
mean,5.653636e+05
std,3.159714e+05
min,8.235377e+04
25%,3.628767e+05
50%,4.889236e+05
75%,6.606955e+05
max,2.839374e+06


## Step 4: Export your predictions

In [510]:
final_df.to_csv('housing_preds_Alex_Zieky.csv',header = None)